In [5]:
# First we import the libraries
import numpy as np
import matplotlib.pyplot as plt

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [6]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta ----------------------------#
delta = "1.10" 
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
#---------------------------------------------------------------#

train_path = "data/nY=8/δ="+delta+"_samples.csv"
train_data = data.load_data(train_path)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

torch.Size([10000, 8])

In [7]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the 
# number of visible nodes (nv) and number of higgen nodes (nh). 
# As we have an array of 8 atoms, we have 6 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the 
# data we are using and which can be varied to get optimal result. 
# For this problem, nh was varied from 4 to 8 and nh = 8 gave quite good results.
# Hence, we set nh = 8.
nh = 8

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [8]:
# Below we have more hyperparameters of the RBM model. Like the number of 
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions. 
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100
nbs = pbs
lr = 0.0065 
k = 10

# Now we train our RBM using the above parameters. 
# Note that the training process will take 2 - 3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

Total time elapsed during training: 84.906 s


In [9]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt in the output directory.
nn_state.save("output/rydberg_data.pt")
torch.load("output/rydberg_data.pt")
# Below we have the parameters  (weights and biases) of the trained RBM:

{'rbm_am': OrderedDict([('weights',
               tensor([[ 1.0773, -1.1303,  0.7778, -0.3806, -0.1517,  0.1679, -0.1082, -0.6292],
                       [-0.0962,  0.5712, -1.3292,  1.2913, -0.9854, -0.0900,  0.5016, -0.6935],
                       [ 0.1845,  0.1810, -0.3647, -0.5876,  1.3849, -2.2735,  2.3106, -1.7423],
                       [-1.2608,  0.4682, -0.4169,  0.0740, -0.8023,  0.9898, -1.7133,  1.5958],
                       [-2.5412,  1.3499, -0.2856, -0.4839,  0.5009,  0.0877, -1.3290,  2.4340],
                       [-0.7942,  1.2317, -2.1229,  1.9389, -1.6194,  0.4200, -0.2122, -0.1168],
                       [ 0.7034, -0.3309, -0.7011,  1.4245, -1.8230,  1.2051, -0.4778, -0.5550],
                       [ 1.1452, -2.0901,  1.6725, -1.7995,  1.0740, -1.1908,  0.3674, -1.0747]],
                      dtype=torch.float64)),
              ('visible_bias',
               tensor([-0.4311, -1.1356,  0.0642, -1.6188, -0.1884, -0.6677, -0.8371, -0.8035],
               

In [10]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM. 
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 50000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('output/reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)
        
# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hencem there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in 
# our produced sample in in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like: 
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>  
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the 
# amplitudes in amplitudeList and then also 
# prints out the amplitudes in 'output/reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (256):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('output/reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

In [11]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and 
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1) 
# in each state.

# Now we get average occupation of each site by finding the 
# number of atoms in the up state in each site and then
# by dividing by the number of the produced samples 
occupation = [0]*8
for i in range(len(sampleList)):
    occupation[0] = sampleList[i][0] + occupation[0]  
    occupation[1] = sampleList[i][1] + occupation[1]
    occupation[2] = sampleList[i][2] + occupation[2]
    occupation[3] = sampleList[i][3] + occupation[3]
    occupation[4] = sampleList[i][4] + occupation[4]
    occupation[5] = sampleList[i][5] + occupation[5]
    occupation[6] = sampleList[i][6] + occupation[6]
    occupation[7] = sampleList[i][7] + occupation[7]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

[0.37546, 0.42556, 0.35986, 0.44446, 0.35518, 0.4222, 0.37734, 0.40718]

In [401]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((8,8))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the 
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected 
# to be nearly equal (which they are)

# Note that the matrix in data/nY=8/δ=delta_2_pt_fn.csv may be a bit misleading. 
# The matrix generated there is symmetric and this is why the values 
# below the diagonal were not computed. However the actual matrix is symmetric
TwoPointFunc

array([[0.37048, 0.02514, 0.24578, 0.06414, 0.22386, 0.05262, 0.25284,
        0.02168],
       [0.02514, 0.4306 , 0.02792, 0.31088, 0.0442 , 0.28422, 0.05808,
        0.29872],
       [0.24578, 0.02792, 0.35432, 0.02602, 0.24626, 0.0463 , 0.22496,
        0.0489 ],
       [0.06414, 0.31088, 0.02602, 0.44488, 0.0252 , 0.30838, 0.06708,
        0.2749 ],
       [0.22386, 0.0442 , 0.24626, 0.0252 , 0.34886, 0.02444, 0.2445 ,
        0.0482 ],
       [0.05262, 0.28422, 0.0463 , 0.30838, 0.02444, 0.4238 , 0.02834,
        0.29732],
       [0.25284, 0.05808, 0.22496, 0.06708, 0.2445 , 0.02834, 0.37274,
        0.02648],
       [0.02168, 0.29872, 0.0489 , 0.2749 , 0.0482 , 0.29732, 0.02648,
        0.41232]])